# Convolutional neural networks for genomics: 
## How to train your DragoNN tutorial

This tutorial is a supplement to the DragoNN manuscript.

## Outline<a name='outline'>
<ol>
    <li><a href=#1>How to use this tutorial</a></li>
    <li><a href=#2>Review of patterns in transcription factor binding sites</a></li>
    <li><a href=#3>Learning to localize homotypic motif density</a></li>
    <li><a href=#4>Simulate training data with simdna</a></li>
    
[The DragoNN software package](https://github.com/kundajelab/dragonn) provides a number of utilities for training convolutional neural networks (CNN's) for genomic sequences. We illustrate how DragoNN can be used to train and interpret a series of increasingly complex CNN's. For more advanced users, we provide the corresponding [keras](http://keras.io) code. 

    
   <li><a href=#5>Model architecture</a></li>
   <li><a href=#6>Model training</a></li>
   <li><a href=#7>Model interpretation</a></li>
   <li><a href=#8>single layer, multiple filter models</a></li>
            <ol>
                <li><a href=#8a>DragoNN implementation</a></li>
                <li><a href=#8b>Keras implementation</a></li>
            </ol>
            <li><a href=#9>Multi-layer models</a></li>
            <ol>
                <li><a href=#9a>DragoNN implementation</a></li>
                <li><a href=#9b>Keras implementation</a></li>
            </ol>
            <li><a href=#10>Regularized multi-layer models</a></li>
            <ol>
                <li><a href=#10a>DragoNN implementation</a></li>
                <li><a href=#10b>Keras implementation</a></li>
            </ol>
    <li><a href=#11>Further exploration</a></li>
    <li><a href=#12>Using DragoNN with your own non-simulated data</a></li>
</ol>
Github issues on the dragonn repository with feedback, questions, and discussion are always welcome.

 

## How to use this tutorial<a name='1'>
<a href=#outline>Home</a>

This tutorial utilizes a Jupyter/IPython Notebook - an interactive computational enviroment that combines live code, visualizations, and explanatory text. The notebook is organized into a series of cells. You can run the next cell by cliking the play button:
![play button](./primer_tutorial_images/play_button.png)
You can also run all cells in a series by clicking "run all" in the Cell drop-down menu:
![play all button](./primer_tutorial_images/play_all_button.png)
Half of the cells in this tutorial contain code, the other half contain visualizations and explanatory text. Code, visualizations, and text in cells can be modified - you are encouraged to modify the code as you advance through the tutorial. You can inspect the implementation of a function used in a cell by following these steps:
![inspecting code](./primer_tutorial_images/inspecting_code.png)

We start by loading dragonn's tutorial utilities and reviewing properties of regulatory sequence that transcription factors bind.

In [23]:
#load dragonn tutorial utilities 
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from dragonn.tutorial_utils import *


## Key properties of regulatory DNA sequences <a name='2'>
<a href=#outline>Home</a>

![sequence properties 1](./primer_tutorial_images/sequence_properties_1.jpg)
![sequence properties 2](./primer_tutorial_images/sequence_properties_2.jpg)

## Learning to localize homotypic motif density <a name='3'>
<a href=#outline>Home</a>

In this tutorial we will learn how to localize a homotypic motif cluster. We will simulate a positive set of sequences with multiple instances of a motif in the center and a negative set of sequences with multiple motif instances positioned anywhere in the sequence:
![homotypic motif density localization](./primer_tutorial_images/homotypic_motif_density_localization.jpg)
We will then train a binary classification model to classify the simulated sequences. To solve this task, the model will need to learn the motif pattern and whether instances of that pattern are present in the central part of the sequence.

![classificatioin task](./tutorial_images/homotypic_motif_density_localization_task.jpg)

We start by getting the simulation data.

## Getting simulation data <a name='4'>
<a href=#outline>Home</a>


DragoNN provides a set of simulation functions. We will use the **simulate_motif_density_localization** function to simulate homotypic motif density localization. First, we obtain documentation for the simulation parameters.

In [4]:
print_simulation_info("simulate_motif_density_localization")


    Simulates two classes of seqeuences:
        - Positive class sequences with multiple motif instances
          in center of the sequence.
        - Negative class sequences with multiple motif instances
          anywhere in the sequence.
    The number of motif instances is uniformly sampled
    between minimum and maximum motif counts.

    Parameters
    ----------
    motif_name : str
        encode motif name
    seq_length : int
        length of sequence
    center_size : int
        length of central part of the sequence where motifs can be positioned
    min_motif_counts : int
        minimum number of motif instances
    max_motif_counts : int
        maximum number of motif instances
    num_pos : int
        number of positive class sequences
    num_neg : int
        number of negative class sequences
    GC_fraction : float
        GC fraction in background sequence

    Returns
    -------
    sequence_arr : 1darray
        Contains sequence strings.
    y : 1darra

Next, we define parameters for a TAL1 motif density localization in 1500bp long sequence, with 0.4 GC fraction, and 2-4 instances of the motif in the central 150bp for the positive sequences. We simulate a total of 3000 positive and 3000 negative sequences.

In [10]:
motif_density_localization_simulation_parameters = {
    "motif_name": "TAL1_known4",
    "seq_length": 1500,
    "center_size": 150,
    "min_motif_counts": 2,
    "max_motif_counts": 4, 
    "num_pos": 3000,
    "num_neg": 3000,
    "GC_fraction": 0.4}

We get the simulation data by calling the **get_simulation_data** function with the simulation name and the simulation parameters as inputs. 1000 sequences are held out for a test set, 1000 sequences for a validation set, and the remaining 4000 sequences are in the training set.

In [6]:
simulation_data = get_simulation_data("simulate_motif_density_localization",
                                      motif_density_localization_simulation_parameters,
                                      validation_set_size=1000, test_set_size=1000)

simulation_data provides training, validation, and test sets of input sequences X and sequence labels y. The inputs X are matrices with a one-hot-encoding of the sequences:
<img src="tutorial_images/one_hot_encoding.png" width="500">

Here are the first 10bp of a sequence in our training data:

In [11]:
simulation_data.X_train[0, :, :, :10]

array([[[0, 0, 0, 0, 1, 1, 0, 1, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0, 0, 1, 0, 1, 0]]], dtype=int32)

We can convert this one-hot-encoded matrix back into a DNA string:

In [22]:
from dragonn.utils import *
get_sequence_strings(simulation_data.X_train)[0][:10]

b'GTTTAATATA'

Let's examine the shape of training, validation, and test matrices: 

In [15]:
print(simulation_data.X_train.shape)
print(simulation_data.y_train.shape)

(4000, 1, 4, 1000)
(4000, 1)


In [16]:
print(simulation_data.X_valid.shape)
print(simulation_data.y_valid.shape)

(1000, 1, 4, 1000)
(1000, 1)


In [17]:
print(simulation_data.X_test.shape)
print(simulation_data.y_test.shape)

(1000, 1, 4, 1000)
(1000, 1)


# Defining the convolutional neural network model architecture  <a name='5'>
<a href=#outline>Home</a>

A locally connected linear unit in a CNN model can represent a PSSM (part a). A sequence PSSM score is obtained by multiplying the PSSM across the sequence, thresholding the PSSM scores, and taking the max (part b). A PSSM score can also be computed by a CNN model with tiled, locally connected linear units, amounting to a convolutional layer with a single convolutional filter representing the PSSM, followed by ReLU thresholding and maxpooling (part c).
![dragonn vs pssm](./tutorial_images/dragonn_and_pssm.jpg)
By utilizing multiple convolutional layers with multiple convolutional filters, CNN's can represent a wide range of sequence features in a compositional fashion:
![dragonn model figure](./tutorial_images/dragonn_model_figure.jpg)

The main DragoNN model class, **SequenceDNN**, provides a simple interface to a range of models and methods to train, test, and interpret CNN's for genomic data. SequenceDNN uses [keras](http://keras.io/), a deep learning library for [TensorFlow](https://github.com/tensorflow/tensorflow),a popular software package for deep learning.

We will train, test, and interpret a series of increasingly complex CNN models via the DragoNN interface.
For users experienced with keras, each DragoNN vignette is accompanied by the equivalent keras code. 

To get a description of the architecture parameters we use the inspect_SequenceDNN function, which outputs documentation for the model class including the architecture parameters:

In [8]:
inspect_SequenceDNN()

Sequence DNN models.

Parameters
----------
seq_length : int, optional
    length of input sequence.
keras_model : instance of keras.models.Sequential, optional
    seq_length or keras_model must be specified.
num_tasks : int, optional
    number of tasks. Default: 1.
num_filters : list[int] | tuple[int]
    number of convolutional filters in each layer. Default: (15,).
conv_width : list[int] | tuple[int]
    width of each layer's convolutional filters. Default: (15,).
pool_width : int
    width of max pooling after the last layer. Default: 35.
L1 : float
    strength of L1 penalty.
dropout : float
    dropout probability in every convolutional layer. Default: 0.
verbose: int
    Verbosity level during training. Valida values: 0, 1, 2.

Returns
-------
Compiled DNN model.

Available methods:



"Available methods" display what can be done with a SequenceDNN model. These include common operations such as training and testing the model, and more complex operations such as extracting insight from trained models. We define a simple DragoNN model with one convolutional layer with one convolutional filter, followed by maxpooling of width 35. 

In [24]:
one_filter_dragonn_parameters = {
    'seq_length': 1000,
    'num_filters': [1],
    'conv_width': [10],
    'num_epochs': 150,
    'pool_width': 35}

we can get a radnomly initialized DragoNN model by calling the **get_SequenceDNN** function with **one_filter_dragonn_parameters** as the input: 

In [30]:
one_filter_dragonn = get_SequenceDNN(one_filter_dragonn_parameters)

ImportError: cannot import name 'TimeDistributedDense'

**The corresponding keras operations are presented below -- if you are not familiar with keras, please continue to <a href=#6>Model Training</a>**


In [ ]:
#import the necessary functions and submodules from keras
from keras.models import Sequential
import keras.layers as kl
from keras.callbacks import EarlyStopping, History

In [ ]:
#Define the model architecture in keras, 

## Model Training <a name='6'>

Next, we evaluate the one_filter_dragonn by loading an already trained model and its training history. In each epoch, the one_filter_dragonn performed a complete pass over the training data, and updated its parameters to minimize the loss, which quantifies the error in the model predictions. After each epoch, the performance metrics for the one_filter_dragonn on the validation data were stored. Training stopped once the loss on the validation stopped improving for multiple consecutive epochs. The performance metrics include balanced accuracy, area under the receiver-operating curve ([auROC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)), are under the precision-recall curve ([auPRC](https://en.wikipedia.org/wiki/Precision_and_recall)), and recall for multiple false discovery rates  (Recall at [FDR](https://en.wikipedia.org/wiki/False_discovery_rate)).

In [ ]:
import warnings
warnings.filterwarnings('ignore')

one_filter_dragonn = SequenceDNN.load("primer_tutorial_models/one_filter_dragonn.arch.json",
                                           "primer_tutorial_models/one_filter_dragonn.weights.h5")
with open('primer_tutorial_models/one_filter_dragonn.train_metrics.pkl', 'rb') as handle:
    one_filter_dragonn.train_metrics = pickle.load(handle,encoding="latin1")
with open('primer_tutorial_models/one_filter_dragonn.valid_metrics.pkl', 'rb') as handle:
    one_filter_dragonn.valid_metrics = pickle.load(handle,encoding="latin1")

Let's look at the training and validation loss history during training, and performance on the test data.

In [ ]:
SequenceDNN_learning_curve(one_filter_dragonn)
test_SequenceDNN(one_filter_dragonn, simulation_data)

A single layer, single filter model gets good performance and doesn't overfit much

Next, let's visualize the filter learned in this model

In [ ]:
interpret_SequenceDNN_filters(one_filter_dragonn, simulation_data)

The filter captures the reverse complemnt of the simulated motif.

# A multi-filter DragoNN model <a name='7'>
Next, we modify the model to have 15 convolutional filters instead of just one filter. How does this model compare to the single filter model?

In [ ]:
multi_filter_dragonn_parameters = {
    'seq_length': 1000,
    'num_filters': [15], ## notice the change from 1 filter to 15 filters
    'conv_width': [10],
    'pool_width': 35}
multi_filter_dragonn = SequenceDNN.load("primer_tutorial_models/multi_filter_dragonn.arch.json",
                                           "primer_tutorial_models/multi_filter_dragonn.weights.h5")
with open('primer_tutorial_models/multi_filter_dragonn.train_metrics.pkl', 'rb') as handle:
    multi_filter_dragonn.train_metrics = pickle.load(handle,encoding="latin1")
with open('primer_tutorial_models/multi_filter_dragonn.valid_metrics.pkl', 'rb') as handle:
    multi_filter_dragonn.valid_metrics = pickle.load(handle,encoding="latin1")
SequenceDNN_learning_curve(multi_filter_dragonn)
test_SequenceDNN(multi_filter_dragonn, simulation_data)

It slightly outperforms the single filter model and exhibits some overfitting. Let's check if the learned filters capture the simulated pattern.

In [ ]:
interpret_SequenceDNN_filters(multi_filter_dragonn, simulation_data)

Only some of the filters closesly match the simulated pattern. This illustrates that interpreting model parameters directly works partially for multi-filter models.

# A multi-layer DragoNN model <a name='8'>
Next, we train a 3 layer model for this task. Will it outperform the single layer model and to what extent will it overfit?

In [ ]:
multi_layer_dragonn_parameters = {
    'seq_length': 1000,
    'num_filters': [15, 15, 15], ## notice the change to multiple filter values, one for each layer
    'conv_width': [10, 10, 10],
    'pool_width': 35}
multi_layer_dragonn = SequenceDNN.load("primer_tutorial_models/multi_layer_dragonn.arch.json",
                                           "primer_tutorial_models/multi_layer_dragonn.weights.h5")
with open('primer_tutorial_models/multi_layer_dragonn.train_metrics.pkl', 'rb') as handle:
    multi_layer_dragonn.train_metrics = pickle.load(handle,encoding="latin1")
with open('primer_tutorial_models/multi_layer_dragonn.valid_metrics.pkl', 'rb') as handle:
    multi_layer_dragonn.valid_metrics = pickle.load(handle,encoding="latin1")
SequenceDNN_learning_curve(multi_layer_dragonn)
test_SequenceDNN(multi_layer_dragonn, simulation_data)

This model performs slightly better than the single layer model but it overfits more. We will try to address that with dropout regularization. But first, what do the first layer filters look like?

In [ ]:
interpret_SequenceDNN_filters(multi_layer_dragonn, simulation_data)

The filters now make less sense than in the single layer model case. In multi-layered models, sequence features are learned compositionally across the layers. As a result, sequence filters in the first layer focus more on simple features that can be combined in higher layers to learn motif features more efficiently, and their interpretation becomes less clear based on simple visualizations.

# A regularized multi-layer DragoNN model <a name='9'>
Next, we regularize the 3 layer using 0.2 dropout on every convolutional layer. Will dropout improve validation performance?

In [ ]:
regularized_multi_layer_dragonn_parameters = {
    'seq_length': 1000,
    'num_filters': [15, 15, 15],
    'conv_width': [10, 10, 10],
    'pool_width': 35,
    'dropout': 0.2} ## we introduce dropout of 0.2 on every convolutional layer for regularization
regularized_multi_layer_dragonn = SequenceDNN.load(
    "primer_tutorial_models/regularized_multi_layer_dragonn.arch.json",
    "primer_tutorial_models/regularized_multi_layer_dragonn.weights.h5")
with open('primer_tutorial_models/regularized_multi_layer_dragonn.train_metrics.pkl', 'rb') as handle:
    regularized_multi_layer_dragonn.train_metrics = pickle.load(handle,encoding="latin1")
with open('primer_tutorial_models/regularized_multi_layer_dragonn.valid_metrics.pkl', 'rb') as handle:
    regularized_multi_layer_dragonn.valid_metrics = pickle.load(handle,encoding="latin1")
SequenceDNN_learning_curve(regularized_multi_layer_dragonn)
test_SequenceDNN(regularized_multi_layer_dragonn, simulation_data)

As expected, dropout decreased the overfitting this model displayed previously and increased test performance. Let's see the learned filters.

In [ ]:
interpret_SequenceDNN_filters(regularized_multi_layer_dragonn, simulation_data)

As in the previous multi layer model, individual filters do not capture the simulated motif.